# LGBM을 활용한 베이스라인

In [1]:
import pandas as pd
import os
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")

## 1. 데이터 로딩

In [140]:
# data_dir = '/opt/ml/input/data/train_dataset'
# csv_file_path = os.path.join(data_dir, 'train_data.csv')
# trainset = pd.read_csv(csv_file_path)
df = pd.read_csv('/opt/ml/code/Lgbm/month_train.csv')
test_df = pd.read_csv('/opt/ml/code/Lgbm/month_test.csv')
# df = pd.read_csv('/opt/ml/code/Lgbm/all_train.csv')
# test_df = pd.read_csv('/opt/ml/code/Lgbm/all_test.csv')

In [100]:
df

,Unnamed: 0,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,month,sec_time,solve_time,...,tag_t_std,tag_t_skew,test_rate,grade,grade_test_level,que_rate,que_num,que_num_rate,tag_rate,user_grade
0,0,0-10,A080096008,A080000096,0,2020-10-01 20:02:27,8931,10,1601582547,55.5,...,142.024193,10.899614,0.71,8,6,0.78,8,0.46,0.55,8
1,1,0-10,A080096001,A080000096,1,2020-10-01 20:04:16,7034,10,1601582656,109.0,...,164.478524,10.210122,0.71,8,6,0.86,1,0.75,0.55,8
2,2,0-10,A080096002,A080000096,1,2020-10-01 20:04:21,8931,10,1601582661,5.0,...,142.024193,10.899614,0.71,8,6,0.56,2,0.72,0.55,8
3,3,0-10,A080096003,A080000096,1,2020-10-01 20:04:33,8931,10,1601582673,12.0,...,142.024193,10.899614,0.71,8,6,0.76,3,0.69,0.55,8
4,4,0-10,A080096004,A080000096,0,2020-10-01 20:04:36,8931,10,1601582676,3.0,...,142.024193,10.899614,0.71,8,6,0.62,4,0.66,0.55,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,2525951,999-9,A020138002,A020000138,0,2020-09-28 06:50:33,8020,9,1601275833,92.0,...,138.612746,9.528800,0.79,2,15,0.86,2,0.72,0.80,8
2525952,2525952,999-9,A020138003,A020000138,1,2020-09-28 06:51:55,8020,9,1601275915,82.0,...,138.612746,9.528800,0.79,2,15,0.89,3,0.69,0.80,8
2525953,2525953,999-9,A020138004,A020000138,1,2020-09-28 06:52:43,8020,9,1601275963,48.0,...,138.612746,9.528800,0.79,2,15,0.84,4,0.66,0.80,8
2525954,2525954,999-9,A020138005,A020000138,0,2020-09-28 06:53:16,8020,9,1601275996,33.0,...,138.612746,9.528800,0.79,2,15,0.77,5,0.60,0.80,8


In [23]:
pd.DataFrame(trainset['userID'].unique().reset_index(name='user'))

,0
0,0
1,1
2,2
3,5
4,6
...,...
6693,7436
6694,7437
6695,7438
6696,7440


In [97]:
df1= trainset

In [98]:
df1['Timestamp']=pd.to_datetime(df1['Timestamp'].values)
df1['month'] = df1['Timestamp'].dt.month
df1['userID'] = (df1['userID'].map(str)+'-'+df1['month'].map(str))
df1.drop(columns=['month'],inplace=True)


train_user_id = df1['userID'].unique()


In [141]:
len(train_user_id)

42228

In [4]:
# df['Timestamp'] = pd.to_datetime(df['Timestamp'].values)
# df['month'] = df['Timestamp'].dt.month
# df['user_id'] = df['userID'].map(str)+'-'+df['month'].map(str)
# df['user_id'] = df['user_id']
# del df['userID']
# df.rename(columns = {'user_id' : 'userID'}, inplace = True)

# df = df[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'month']]

In [5]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,month
0,0-3,A060001001,A060000001,1,2020-03-24 00:17:11,7224,3
1,0-3,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3
2,0-3,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3
3,0-3,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3
4,0-3,A060001005,A060000001,1,2020-03-24 00:17:36,7225,3
...,...,...,...,...,...,...,...
2266581,7441-6,A030071005,A030000071,0,2020-06-05 06:50:21,438,6
2266582,7441-8,A040165001,A040000165,1,2020-08-21 01:06:39,8836,8
2266583,7441-8,A040165002,A040000165,1,2020-08-21 01:06:50,8836,8
2266584,7441-8,A040165003,A040000165,1,2020-08-21 01:07:36,8836,8


In [76]:
train_user_id = df['userID'].unique()
train_user_id

array(['0-10', '0-11', '0-12', ..., '999-7', '999-8', '999-9'],
      dtype=object)

In [109]:
len(train_user_id)

42228

### test data, train에 추가하기(concat)

In [7]:
test_train = pd.read_csv('/opt/ml/input/data/train_dataset/test_data.csv')
test_train = test_train[test_train['answerCode'] != -1]

In [8]:
test_train['Timestamp'] = pd.to_datetime(test_train['Timestamp'].values)
test_train['month'] = test_train['Timestamp'].dt.month
test_train['user_id'] = test_train['userID'].map(str)+'-'+test_train['month'].map(str)
test_train['user_id'] = test_train['user_id']
del test_train['userID']
test_train.rename(columns = {'user_id' : 'userID'}, inplace = True)

test_train = test_train[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'month']]

In [9]:
df = pd.concat([df,test_train])

df.reset_index(drop=True, inplace=True)

df.sort_values(by=['userID','Timestamp'], axis=0)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,month
568,0-10,A080096008,A080000096,0,2020-10-01 20:02:27,8931,10
569,0-10,A080096001,A080000096,1,2020-10-01 20:04:16,7034,10
570,0-10,A080096002,A080000096,1,2020-10-01 20:04:21,8931,10
571,0-10,A080096003,A080000096,1,2020-10-01 20:04:33,8931,10
572,0-10,A080096004,A080000096,0,2020-10-01 20:04:36,8931,10
...,...,...,...,...,...,...,...
693938,999-9,A020138002,A020000138,0,2020-09-28 06:50:33,8020,9
693939,999-9,A020138003,A020000138,1,2020-09-28 06:51:55,8020,9
693940,999-9,A020138004,A020000138,1,2020-09-28 06:52:43,8020,9
693941,999-9,A020138005,A020000138,0,2020-09-28 06:53:16,8020,9


## 2. Feature Engineering

In [10]:
# 문항 번호별 정답률 feature
def que_num_rate(df) :
    
    train_df = df.groupby(['que_num'])['que_num'].count().reset_index(name='que_num_cnt')
    train_ans_df = df.groupby(['que_num'])['answerCode'].sum().reset_index(name='que_ans_cnt')
    train_df = train_df.merge(train_ans_df, on ='que_num', how='left')
    train_df['que_num_rate'] = round(train_df['que_ans_cnt'] / train_df['que_num_cnt'], 2)

    return train_df.loc[:,['que_num','que_num_rate']]


In [11]:
# last Tag에 대한 정답률 함수 생성
def last_tag_rate(df):
    df = df.copy()
    
    last_tag = df.groupby(['userID']).tail(1)
    last_tag_1 = last_tag.loc[:, ['userID','KnowledgeTag']]
    
    user_tag_ans = last_tag_1.merge(df, on=['userID','KnowledgeTag'], how='left')
    
    user_tag_ans['count'] = user_tag_ans.groupby(['userID','KnowledgeTag'])['KnowledgeTag'].transform('count')
    user_tag_ans['tag_cnt'] = user_tag_ans['count']-1
    user_tag_ans['ans_cnt'] = user_tag_ans.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    
    user_tag_ans_rate = user_tag_ans.groupby('userID').tail(1)
    user_tag_ans_rate['ans_cnt'] = user_tag_ans_rate['ans_cnt'].fillna(0)
    user_tag_ans_rate['last_ans_rate'] = round(user_tag_ans_rate['ans_cnt']/ user_tag_ans_rate['tag_cnt'],2)
    user_tag_ans_rate['last_ans_rate'] = user_tag_ans_rate['last_ans_rate'].fillna(0.00)
    
    return user_tag_ans_rate.loc[:,['userID','last_ans_rate']]

In [12]:
def user_test_rate(df):
#     df['index'] = df.index
    u_test_cnt = df.groupby(['userID','testId'])['testId'].cumcount().reset_index(name='u_test_cnt')
    user_test = df.merge(u_test_cnt, on='index', how='left')
    u_test_cnt= user_test.groupby(['userID','testId'])['answerCode'].transform(lambda x: x.cumsum().shift(1)).reset_index(name='test_ans_cnt')
    user_test_ans_sum =  user_test.merge(u_test_cnt, on='index', how='left')
    user_test_ans_sum['test_ans_cnt'] = user_test_ans_sum['test_ans_cnt'].fillna(0.0)
    
    def rating_1(user_test_ans_sum):
        if user_test_ans_sum['u_test_cnt'] == 0:
            return 0.50
        else :    
            return round(user_test_ans_sum['test_ans_cnt']/user_test_ans_sum['u_test_cnt'],2)
        
    user_test_ans_sum['test_ans_rate'] = user_test_ans_sum.apply(rating_1, axis=1)
    
    return user_test_ans_sum.loc[:,['index','u_test_cnt','test_ans_cnt','test_ans_rate']]

In [12]:
# user별 Tag에 대한 정답률 sequential하게 적용
def user_tag_rate(df) :
#     df['index'] = df.index

    u_t_cnt = df.groupby(['userID','KnowledgeTag'])['KnowledgeTag'].cumcount().reset_index(name='u_tag_cnt')
    user_tag =  df.merge(u_t_cnt, on='index', how='left')
    #user_tag['tag_cnt_1'] = user_tag['u_tag_cnt']+1

    u_t_cnt= user_tag.groupby(['userID','KnowledgeTag'])['answerCode'].transform(lambda x: x.cumsum().shift(1)).reset_index(name='ans_cnt')
    user_tag_ans_sum =  user_tag.merge(u_t_cnt, on='index', how='left')
    user_tag_ans_sum['ans_cnt'] = user_tag_ans_sum['ans_cnt'].fillna(0.0)

    def rating(user_tag_ans_sum):
        if user_tag_ans_sum['u_tag_cnt'] == 0:
            return 0.50
        else :    
            return round(user_tag_ans_sum['ans_cnt']/user_tag_ans_sum['u_tag_cnt'],2)
    user_tag_ans_sum['tag_ans_rate'] = user_tag_ans_sum.apply(rating, axis=1)
    
    return user_tag_ans_sum.loc[:,['index','u_tag_cnt','ans_cnt','tag_ans_rate']]

In [13]:
def test_rate(df):
    test_df = df.groupby(['testId'])['testId'].count().reset_index(name='test_cnt')
    test_ans_df = df.groupby(['testId'])['answerCode'].sum().reset_index(name='test_ans_cnt')
    test_df = test_df.merge(test_ans_df, on ='testId', how='left')
    test_df['test_rate'] = round(test_df['test_ans_cnt'] / test_df['test_cnt'], 2)
    test_df['test_rate'] = test_df['test_rate'].fillna(0.00)
    return test_df.loc[:,['testId','test_rate']]

def que_rate(df):
    que_df = df.groupby(['assessmentItemID'])['assessmentItemID'].count().reset_index(name='que_cnt')
    que_ans_df = df.groupby(['assessmentItemID'])['answerCode'].sum().reset_index(name='que_ans_cnt')
    que_df = que_df.merge(que_ans_df, on ='assessmentItemID', how='left')
    que_df['que_rate'] = round(que_df['que_ans_cnt'] / que_df['que_cnt'], 2)
    que_df['que_rate'] = que_df['que_rate'].fillna(0.00)
    return que_df.loc[:,['assessmentItemID','que_rate']]

def tag_rate(df):
    tag_df = df.groupby(['KnowledgeTag'])['KnowledgeTag'].count().reset_index(name='tag_cnt')
    tag_ans_df = df.groupby(['KnowledgeTag'])['answerCode'].sum().reset_index(name='tag_ans_cnt')
    tag_df = tag_df.merge(tag_ans_df, on ='KnowledgeTag', how='left')
    tag_df['tag_rate'] = round(tag_df['tag_ans_cnt'] / tag_df['tag_cnt'], 2)
    tag_df['tag_rate'] = tag_df['tag_rate'].fillna(0.00)
    return tag_df.loc[:,['KnowledgeTag','tag_rate']]

In [14]:
def rating_1(df):
    if df['u_test_cnt'] == 0:
        return 0.50
    else :    
        return round(df['test_ans_cnt']/df['u_test_cnt'],2)
    
def rating_2(df):
    if df['u_tag_cnt'] == 0:
        return 0.50
    else :    
        return round(df['tag_ans_cnt']/df['u_tag_cnt'],2)

In [15]:
def feature_engineering(df):
    
    df['index'] = df.index
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']
    
    # user_test_ans_rate
    df['u_test_cnt'] = df.groupby(['userID','testId'])['answerCode'].cumcount()
    df['test_ans_cnt'] = df.groupby(['userID','testId'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['test_ans_cnt'] = df['test_ans_cnt'].fillna(0.0)
    df['test_ans_rate'] = df.apply(rating_1, axis=1)
    
    # user_tag_seq_ans_rate
    df['u_tag_cnt'] = df.groupby(['userID','KnowledgeTag'])['answerCode'].cumcount()
    df['tag_ans_cnt'] = df.groupby(['userID','KnowledgeTag'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['tag_ans_cnt'] = df['tag_ans_cnt'].fillna(0.0)
    df['tag_ans_rate'] = df.apply(rating_2, axis=1)   
    return df

In [16]:
df = feature_engineering(df)
df.head()

,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,month,userID,index,user_correct_answer,user_total_answer,user_acc,u_test_cnt,test_ans_cnt,test_ans_rate,u_tag_cnt,tag_ans_cnt,tag_ans_rate
568,A080096008,A080000096,0,2020-10-01 20:02:27,8931,10,0-10,568,NaN,0,NaN,0,0.0,0.50,0,0.0,0.50
569,A080096001,A080000096,1,2020-10-01 20:04:16,7034,10,0-10,569,0.0,1,0.000000,1,0.0,0.00,0,0.0,0.50
570,A080096002,A080000096,1,2020-10-01 20:04:21,8931,10,0-10,570,1.0,2,0.500000,2,1.0,0.50,1,0.0,0.00
571,A080096003,A080000096,1,2020-10-01 20:04:33,8931,10,0-10,571,2.0,3,0.666667,3,2.0,0.67,2,1.0,0.50
572,A080096004,A080000096,0,2020-10-01 20:04:36,8931,10,0-10,572,3.0,4,0.750000,4,3.0,0.75,3,2.0,0.67


In [17]:
def grade_test_level(df):
    df = test_rate(df)
    df['grade'] = df['testId'].str.slice(start=2, stop=3)
    df['grade_test_level'] = df.groupby(['grade'])['test_rate'].rank(method='dense', ascending=False)
    df['grade_test_level'] = df['grade_test_level'].astype(int)
    return df


In [18]:
grade_test_level = grade_test_level(df)
grade_test_level

,testId,test_rate,grade,grade_test_level
0,A010000001,0.92,1,4
1,A010000002,0.93,1,3
2,A010000003,0.84,1,12
3,A010000004,0.88,1,8
4,A010000005,0.85,1,11
...,...,...,...,...
1532,A090000070,0.46,9,12
1533,A090000071,0.34,9,23
1534,A090000072,0.45,9,13
1535,A090000073,0.54,9,6


In [19]:
df = df.merge(grade_test_level, on='testId', how='left')
df['grade'] = df['grade'].astype(int)
df['user_correct_answer'] = df['user_correct_answer'].fillna(0)

# test_rate
all_test_rate = test_rate(df)
# df = pd.merge(df, all_test_rate, on='testId', how='left')

# que_rate
all_que_rate = que_rate(df)
df = pd.merge(df, all_que_rate, on='assessmentItemID', how='left')

# que_num_rate
df['que_num'] = df['assessmentItemID'].str.slice(start=7, stop=10)
df['que_num'] = df['que_num'].astype(int)
que_num_df = que_num_rate(df)
df = df.merge(que_num_df, on='que_num', how='left')

# tag_rate
all_tag_rate = tag_rate(df)
df = pd.merge(df, all_tag_rate, on='KnowledgeTag', how='left')

In [20]:
# 대표 grade값 매기기
def user_grade(df):
    df_train_grade = df.groupby(['userID','grade'])['grade'].count().reset_index(name='grade_cnt')
    train_grade_last = df_train_grade.groupby(['userID'], as_index=False).max()  # 가장 많은 cnt개수 
    train_grade_last['user_grade'] = train_grade_last['grade']
    all_user_grade = train_grade_last.loc[:,['userID','user_grade']]
    return all_user_grade

In [21]:
all_user_grade = user_grade(df)
df = df.merge(all_user_grade,on='userID', how='left')

## 4. model_prams

In [22]:
model_params = {
        'objective': 'binary', # 이진 분류
        'boosting_type': 'gbdt',
        'metric':'auc',
        'num_leaves': 107,
#         'max_bin': 204,
        # min_data_in_leaf 값을 10-40까지 정수값 중에 사용
        'min_data_in_leaf': 20,
        # 피처 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'feature_fraction': 0.46948747908887395,         # optuna 추천 - 0.46948747908887395
        # 데이터 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'bagging_fraction': 0.9,
        # 데이터 샘플링 횟수를 1-7까지 정수값 중에 사용
        'bagging_freq': 1,
#         'n_estimators': 100, # 트리 개수
        'early_stopping_rounds': 100,
        # L1 값을 1e-8-10.0까지 로그 uniform 분포로 사용
        'lambda_l1': 0.00010175824628083187,       # 0.00010175824628083187
        # L2 값을 1e-8-10.0까지 로그 uniform 분포로 사용
        'lambda_l2': 0.0001150682989588032,    # 0.0001150682989588032
        'seed': 42,
        'verbose': -1,
        'n_jobs': -1,    
    }

In [23]:
model_params = {
        'objective': 'binary', # 이진 분류
        'boosting_type': 'gbdt',
        'metric':'auc',
        'num_leaves': 227,
#         'max_bin': 204,
        # min_data_in_leaf 값을 10-40까지 정수값 중에 사용
        'min_data_in_leaf': 11,
        # 피처 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'feature_fraction': 0.647794747944201,         # optuna 추천 - 0.46948747908887395
        # 데이터 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'bagging_fraction': 0.9,
        # 데이터 샘플링 횟수를 1-7까지 정수값 중에 사용
        'bagging_freq': 4,   # 1
#         'n_estimators': 100, # 트리 개수
        'early_stopping_rounds': 100,
        # L1 값을 1e-8-10.0까지 로그 uniform 분포로 사용
        'lambda_l1': 8.755042100872112e-06,       # 0.00010175824628083187
        # L2 값을 1e-8-10.0까지 로그 uniform 분포로 사용
        'lambda_l2': 0.0015429233553149078,    # 0.0001150682989588032
        'seed': 42,
        'verbose': -1,
        'n_jobs': -1,    
    }

In [101]:
model_params = {
        'objective': 'binary', # 이진 분류
        'boosting_type': 'gbdt',
        'metric':'auc',
        'num_leaves': 100,    #  256,
#         'max_bin': 204,
        # min_data_in_leaf 값을 10-40까지 정수값 중에 사용
        'min_data_in_leaf': 27,   # 38,
        # 피처 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'feature_fraction':0.6623969601759928,   # 0.6072955014217922,   # 0.6623969601759928,  # 0.46948747908887395
        # 데이터 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
        'bagging_fraction': 0.8420665969537324,   # 0.5877385312344033,
        # 데이터 샘플링 횟수를 1-7까지 정수값 중에 사용
        'bagging_freq': 2,   # 1
#         'n_estimators': 100, # 트리 개수
        'early_stopping_rounds': 100,
#         # L1 값을 1e-8-10.0까지 로그 uniform 분포로 사용
#         'lambda_l1': 2.8559537699120116e-06,    #8.755042100872112e-06,       # 0.00010175824628083187
#         # L2 값을 1e-8-10.0까지 로그 uniform 분포로 사용
#         'lambda_l2': 2.2432872572212683e-06,   # 0.0015429233553149078,    # 0.0001150682989588032
        'seed': 42,   #37
        'verbose': -1,
        'n_jobs': -1,    
    }

## 5. Train & Inference

In [24]:
# LOAD TESTDATA
test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
test_df = pd.read_csv(test_csv_file_path)

# FEATURE ENGINEERING
test_df = feature_engineering(test_df)

In [26]:
test_df = test_df.merge(grade_test_level, on='testId', how='left')
test_df['grade'] = test_df['grade'].astype(int)

test_df['user_correct_answer'] = test_df['user_correct_answer'].fillna(0)

test_df = pd.merge(test_df, all_que_rate, on='assessmentItemID', how='left')

# que_num_rate
test_df['que_num'] = test_df['assessmentItemID'].str.slice(start=7, stop=10)
test_df['que_num'] = test_df['que_num'].astype(int)

test_df = test_df.merge(que_num_df, on='que_num', how='left')

# tag_rate
test_df = pd.merge(test_df, all_tag_rate, on='KnowledgeTag', how='left')

In [108]:
test_df

,Unnamed: 0,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,sec_time,solve_time,test_level_diff_itemID,test_level_diff_KnowledgeTag,...,tag_sum,test_t_mean,test_t_std,test_t_skew,assess_t_mean,assess_t_std,assess_t_skew,tag_t_mean,tag_t_std,tag_t_skew
1035,1035,3,A050133008,A050000133,2020-10-26 13:13:57,5289,1603718037,46.0,0.265060,0.244383,...,1818.0,97.591012,205.938841,7.834966,97.569612,135.546503,4.010133,120.805222,256.594927,7.025663
1706,1706,4,A070146008,A070000146,2020-12-27 02:47:54,9080,1609037274,23.0,0.234483,0.181747,...,1389.0,85.117225,191.851553,8.134391,82.013793,269.559311,6.245555,61.873965,164.741240,8.863346
3023,3023,13,A070111008,A070000111,2020-12-27 04:35:09,9660,1609043709,8.0,0.112450,0.136849,...,1481.0,62.649617,220.720211,9.368558,42.692771,165.377741,12.243470,67.534397,188.932400,8.627428
4283,4283,17,A090064006,A090000064,2020-10-30 05:48:37,2611,1604036917,75.0,0.121212,0.141343,...,356.0,57.233564,190.290802,11.571533,63.893939,208.571016,6.065930,51.741349,177.397677,11.343797
4670,4670,26,A060135007,A060000135,2020-10-23 11:44:18,1422,1603453458,17.0,0.208835,0.267004,...,2895.0,81.624490,193.678897,7.521587,86.040161,211.298020,9.915499,87.485918,200.808826,7.995881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260052,260052,7395,A040122005,A040000122,2020-09-08 02:05:20,10615,1599530720,2.0,0.227425,0.238209,...,1473.0,64.410628,164.028851,10.563012,52.077832,82.488142,5.088637,61.338983,160.712269,8.847763
260067,260067,7404,A030111005,A030000111,2020-10-13 09:49:18,7636,1602582558,107.0,0.307692,0.299646,...,3957.0,60.397586,174.624862,9.427831,41.287625,145.748603,13.286456,59.856745,148.740595,11.440565
260082,260082,7416,A050193004,A050000193,2020-10-04 02:44:41,10402,1601779481,24.0,0.282258,0.334230,...,2755.0,42.642786,93.702788,8.514062,52.153226,115.624658,9.786879,50.662481,138.930080,11.483601
260097,260097,7417,A050193004,A050000193,2020-09-06 13:09:15,10402,1599397755,21.0,0.282258,0.334230,...,2755.0,42.642786,93.702788,8.514062,52.153226,115.624658,9.786879,50.662481,138.930080,11.483601


In [142]:
# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(['answerCode'], axis=1)

test_df

,Unnamed: 0,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,sec_time,solve_time,test_level_diff_itemID,test_level_diff_KnowledgeTag,...,tag_sum,test_t_mean,test_t_std,test_t_skew,assess_t_mean,assess_t_std,assess_t_skew,tag_t_mean,tag_t_std,tag_t_skew
1035,1035,3,A050133008,A050000133,2020-10-26 13:13:57,5289,1603718037,46.0,0.265060,0.244383,...,1818.0,97.591012,205.938841,7.834966,97.569612,135.546503,4.010133,120.805222,256.594927,7.025663
1706,1706,4,A070146008,A070000146,2020-12-27 02:47:54,9080,1609037274,23.0,0.234483,0.181747,...,1389.0,85.117225,191.851553,8.134391,82.013793,269.559311,6.245555,61.873965,164.741240,8.863346
3023,3023,13,A070111008,A070000111,2020-12-27 04:35:09,9660,1609043709,8.0,0.112450,0.136849,...,1481.0,62.649617,220.720211,9.368558,42.692771,165.377741,12.243470,67.534397,188.932400,8.627428
4283,4283,17,A090064006,A090000064,2020-10-30 05:48:37,2611,1604036917,75.0,0.121212,0.141343,...,356.0,57.233564,190.290802,11.571533,63.893939,208.571016,6.065930,51.741349,177.397677,11.343797
4670,4670,26,A060135007,A060000135,2020-10-23 11:44:18,1422,1603453458,17.0,0.208835,0.267004,...,2895.0,81.624490,193.678897,7.521587,86.040161,211.298020,9.915499,87.485918,200.808826,7.995881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260052,260052,7395,A040122005,A040000122,2020-09-08 02:05:20,10615,1599530720,2.0,0.227425,0.238209,...,1473.0,64.410628,164.028851,10.563012,52.077832,82.488142,5.088637,61.338983,160.712269,8.847763
260067,260067,7404,A030111005,A030000111,2020-10-13 09:49:18,7636,1602582558,107.0,0.307692,0.299646,...,3957.0,60.397586,174.624862,9.427831,41.287625,145.748603,13.286456,59.856745,148.740595,11.440565
260082,260082,7416,A050193004,A050000193,2020-10-04 02:44:41,10402,1601779481,24.0,0.282258,0.334230,...,2755.0,42.642786,93.702788,8.514062,52.153226,115.624658,9.786879,50.662481,138.930080,11.483601
260097,260097,7417,A050193004,A050000193,2020-09-06 13:09:15,10402,1599397755,21.0,0.282258,0.334230,...,2755.0,42.642786,93.702788,8.514062,52.153226,115.624658,9.786879,50.662481,138.930080,11.483601


In [143]:
test_df.columns

Index(['Unnamed: 0', 'userID', 'assessmentItemID', 'testId', 'Timestamp',
       'KnowledgeTag', 'sec_time', 'solve_time', 'test_level_diff_itemID',
       'test_level_diff_KnowledgeTag', 'test_level_diff_testId', 'index',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'u_test_cnt',
       'test_ans_cnt', 'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt',
       'tag_ans_rate', 'test_rate', 'grade', 'grade_test_level', 'que_rate',
       'que_num', 'que_num_rate', 'tag_rate', 'wrongP_time', 'correctP_time',
       'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean',
       'tag_sum', 'test_t_mean', 'test_t_std', 'test_t_skew', 'assess_t_mean',
       'assess_t_std', 'assess_t_skew', 'tag_t_mean', 'tag_t_std',
       'tag_t_skew'],
      dtype='object')

In [110]:
import os,gc
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, accuracy_score

#폴드 개수만큼 user를 쪼갠 목록을 만든다
def lgbm_oof_split_data_withidx(df, n_fold):
    #grade별로 user list 생성 (9개)
    u_g = df.groupby(['userID','user_grade'])['userID'].count().reset_index(name='cnt')   #user별 grade값
    users = [[] for _ in range(10)]  #grade개수총 9개
    for id in u_g['userID']:
        id_grade = int(u_g[u_g['userID'] == id]['user_grade'])  #해당 유저의 grade
        id_cnt = int(u_g[u_g['userID'] == id]['cnt'])
        users[id_grade].append((id, id_cnt))
    # grade별로 sorted
    for i in range(1,10):
        user_list = users[i]
        user_list = sorted(user_list, key =lambda x: x[1], reverse=True)
        users[i] = user_list
        
    n_fold = n_fold
    random.seed(42)
    user_id_dict=defaultdict(list)
    user_count_dict=defaultdict(int)
    for i in range(1,10):
        gul = users[i]           # gul = grade user list
        random.shuffle(gul)
        for idx, (id, cnt) in enumerate(gul):
            f_num=idx%n_fold
            user_count_dict[f_num] += cnt
            user_id_dict[f_num].append(id)
    return user_id_dict, user_count_dict

In [111]:
#dict들을 통해 원하는 폴드의 train/val set을 가져온다
def get_fold_data(idx, data, user_id_dict,user_count_dict, train_user_id):
    train = data[data['userID'].isin(user_id_dict[idx]) == False]  # train
    valid = data[data['userID'].isin(user_id_dict[idx])] # validation
    
    #test데이터셋은 각 유저의 마지막 interaction만 추출
    df_last = valid['userID'] != valid['userID'].shift(-1)  #validation
    train_UID = valid['userID'].isin(train_user_id)

    oof_valid = valid[df_last & train_UID].copy()
    
    valid.drop(valid[df_last & train_UID].index, inplace=True)
    
    train = pd.concat([train, valid])
    
    return train, oof_valid

In [112]:
#FE를 진행한 전체 train set과 FE를 진행한 전체 test set, featrue 들을 넣어주시면 됩니다
def make_lgb_user_oof_prediction(train, FEATS, train_user_id, test, model_params=None, folds=5):  # categorical_features='auto',
#     train.shape[0]  # train 행 총 개수
    # fold 생성
    user_id_dict,user_count_dict = lgbm_oof_split_data_withidx(train, folds)
    
    # test data
    x_test = test[FEATS] # 마지막 interaction의 features 값
    
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])
    test_df = test['userID'].to_frame().copy()
    
    # Out Of Fold Validation 예측 데이터를 저장할 변수 (예측값)
    oof_df = pd.DataFrame()
    
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0
    acc=0
    
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = FEATS
    
    for fold in range(folds):
        # train index, validation index로 train 데이터를 나눔
        train_set, valid_set= get_fold_data(fold, train, user_id_dict, user_count_dict, train_user_id)  #개수 잘 나뉘었다.
        
        x_tr, x_val = train_set[FEATS], valid_set[FEATS]    # 정답 외 feature 값들 (마지막 interaction만 X)
        y_tr, y_val = train_set['answerCode'], valid_set['answerCode']  # 정답 부분
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')
        
        # LightGBM 데이터셋 선언
        dtrain = lgb.Dataset(x_tr, label=y_tr)
        dvalid = lgb.Dataset(x_val, label=y_val)
        
        # LightGBM 모델 훈련
        model = lgb.train(
            model_params,
            dtrain,
            valid_sets=[dtrain, dvalid], # Validation 성능을 측정할 수 있도록 설정
            categorical_feature='auto',
            verbose_eval=100,    
            num_boost_round=5000,
        )
        
        # Validation 데이터 예측
        val_preds = model.predict(x_val)
        
        # oof validation 예측값 저장 
        valid_set['preds'] = val_preds
        
        oof_df = pd.concat([oof_df, valid_set[['userID', 'preds','answerCode']]])

        # 폴드별 Validation 스코어 측정
        fold_acc = accuracy_score(y_val, list(map(round,val_preds)))
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)} | ACC: {fold_acc}")
        print('-'*80)
        
        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds
        acc+=fold_acc / folds
        
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += model.predict(x_test) / folds

        
        # 폴드별 피처 중요도 저장
        fi[f'fold_{fold+1}'] = model.feature_importance()

        del x_tr, x_val, y_tr, y_val
        gc.collect()
        
    print(f"\nMean AUC = {score}") # 폴드별 Validation 스코어 출력

    # 폴드별 피처 중요도 평균값 계산해서 저장 
    fi_cols = [col for col in fi.columns if 'fold_' in col]
    fi['importance'] = fi[fi_cols].mean(axis=1)

    # make test_df
    test_df['preds'] = test_preds
    
    return oof_df, test_df

In [144]:
# FEATS = ['user_acc', 'u_test_cnt', 'test_ans_rate', 'u_tag_cnt', 'tag_ans_rate', 'test_rate', 'grade', 'grade_test_level', 'que_rate', 'que_num_rate', 'tag_rate']
# FEATS = [
# #     'sec_time', 'solve_time', 
#         'test_level_diff_itemID',
#        'test_level_diff_KnowledgeTag', 'test_level_diff_testId', 
#        'user_correct_answer', 'user_total_answer', 
#     'user_acc', 
# #        'test_ans_cnt', 
#     'test_ans_rate', 
# #     'u_tag_cnt', 'tag_ans_cnt',
#        'tag_ans_rate', 
# #     'test_rate', 'que_rate',
# #        'que_num', 'que_num_rate', 'tag_rate'
# #     'wrongP_time', 'correctP_time',
# #        'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean',
# #        'tag_sum', 'test_t_mean', 'test_t_std', 'test_t_skew', 'assess_t_mean',
# #        'assess_t_std', 'assess_t_skew', 'tag_t_mean', 'tag_t_std','tag_t_skew'
#         ]

# FEATS = [
# #     'sec_time', 'solve_time', 'test_level_diff_itemID',
# #        'test_level_diff_KnowledgeTag', 'test_level_diff_testId',
#        'user_correct_answer', 'user_total_answer', 'user_acc', 
#         'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt',
#        'tag_ans_rate', 'test_rate', 'que_rate',
#        'que_num', 'que_num_rate', 'tag_rate'
# #     'wrongP_time', 'correctP_time',
# #        'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean',
# #        'tag_sum', 'test_t_mean', 'test_t_std', 'test_t_skew', 'assess_t_mean',
# #        'assess_t_std', 'assess_t_skew', 'tag_t_mean', 'tag_t_std','tag_t_skew'
#         ]

# FEATS = [
# 'sec_time', 'solve_time',
# 'test_level_diff_itemID',
# 'test_level_diff_KnowledgeTag', 'test_level_diff_testId',
# 'wrongP_time', 'correctP_time'
# ]

FEATS = [
'sec_time', 'solve_time',
'wrongP_time', 'correctP_time',
   'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean',
   'tag_sum', 'test_t_mean', 'test_t_std', 'test_t_skew', 'assess_t_mean',
   'assess_t_std', 'assess_t_skew', 'tag_t_mean', 'tag_t_std','tag_t_skew'
    ]

In [145]:
oof_df, test_df = make_lgb_user_oof_prediction(df, FEATS, train_user_id, test_df, model_params=model_params, folds=5 )

fold: 1, x_tr.shape: (2517489, 19), x_val.shape: (8467, 19)
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.821957	valid_1's auc: 0.80696
[200]	training's auc: 0.824953	valid_1's auc: 0.808572
[300]	training's auc: 0.827436	valid_1's auc: 0.809535
[400]	training's auc: 0.82959	valid_1's auc: 0.810157
[500]	training's auc: 0.831469	valid_1's auc: 0.810407
[600]	training's auc: 0.83347	valid_1's auc: 0.811183
[700]	training's auc: 0.83522	valid_1's auc: 0.811293
[800]	training's auc: 0.836952	valid_1's auc: 0.811322
[900]	training's auc: 0.838575	valid_1's auc: 0.811549
[1000]	training's auc: 0.840128	valid_1's auc: 0.811889
[1100]	training's auc: 0.841617	valid_1's auc: 0.811939
Early stopping, best iteration is:
[1089]	training's auc: 0.841454	valid_1's auc: 0.812007
Fold 1 | AUC: 0.8120067018614856 | ACC: 0.7349710641313334
--------------------------------------------------------------------------------
fold: 2, x_tr.shape: (2517499, 19), x_val.sh

In [146]:
oof_df.sort_values(['userID']).to_csv('./valid_preds.csv', index=False)

In [147]:
test_df.to_csv('./test_preds.csv', index=False)

In [128]:
oof_df['userID']

176          0-12
646           0-8
777           1-1
1730        10-10
1778         10-3
            ...  
2524670    998-10
2524734     998-6
2525156    999-11
2525221     999-2
2525729     999-7
Name: userID, Length: 42228, dtype: object

In [44]:
for ele in oof_df['userID']:
    if int(ele.split('-')[0]) == 7441:
        print(ele)

7441-6
7441-8


In [36]:
test_df

,userID,preds
1035,3,0.704978
1706,4,0.767183
3023,13,0.253499
4283,17,0.691950
4670,26,0.248024
...,...,...
260052,7395,0.082474
260067,7404,0.764821
260082,7416,0.841022
260097,7417,0.751179


In [37]:
len(df['userID'].unique())

46996